## Using a pre-trained network

VGG, ResNet, Inception, Xception

** There are two ways to use a pretrained network. Feature Extraction and Fine-Tuning

## Method 1 Feature Extraction

Using representation learned by a previous network to extract interesting features from new samples. These features are then run thru a new classifier which is trained from scratch

Use ConV base from previous trained model and then training a new classifier on top of it

List of all image classification models pretrained on ImageNet dataset
1. Xception
2. Inception V3
3. VGG16
4. Vgg 19


Lets instantiate VGG 16 model

In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


In [3]:
conv_base= VGG16(weights='imagenet',include_top=False, input_shape=(150,150,3))

In [5]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

** The final feature has shape (4,4,512). That's the feature on top of which you will add a densely connected layer

##### Adding a densely connected layer on top of conv base

In [6]:
from keras import models, layers

In [7]:
model= models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
    

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 18,910,017
Trainable params: 18,910,017
Non-trainable params: 0
_________________________________________________________________


*** We have to freeze the conv base. Freezeing a layer meaning avoiding its weight to change during training. If we dont do this, representation that were previously learned by conv base will be modified during training

In [9]:
conv_base.trainable=False

In [10]:
len(model.trainable_weights)

4

### Training a model with frozen conv base

## DON'T RUN THIS ON CPU

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history=model.fit_generator(train_generator,steps_per_epoch=100,epochs=30,validation_data=validation_generator,validation_steps=50)

### Plot results

## Method 2 Fine Tuning

unfreezing a few top layers of frozen model base and jointly train both newly added part of the model( fully connected) and these top layers

CONV 1- frozen
CONV 2- frozen
CONV 3- frozen
CONV 4- we fine tune Conv block 4
CONV 5- we fine tune fully connected classifier

** Steps
1. Add your custom network on top of already trained base network
2. Freeze base network
3. train the part you added
Already done till here above
4. Unfreeze the network
5. Jointly train both these layers and the part you added

In [12]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

We will fine tune these layers, meaning all the remaining should be frozen

block5_conv1 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 9, 9, 512)         2359808   

### Freezing all the layers upto a specific one

In [13]:
conv_base.trainable= True


In [14]:
set_trainable=False
for layer in conv_base.layers:
    if layer.name =='block5_conv1':
        set_trainable=True
    if set_trainable:
        layer.trainable= True
    else:
        layer.trainable= False
        

In [15]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
history=model.fit_generator(train_generator,steps_per_epoch=100,epochs=30,validation_data=validation_generator,validation_steps=50)

In [ ]:
#plot the results